In [46]:
# For compatibility
from __future__ import absolute_import
from __future__ import print_function
# For manipulating data
import pandas as pd
import numpy as np
from keras.utils import np_utils # For y values
# For plotting
%matplotlib inline
import seaborn as sns
# For Keras
from keras.models import Sequential
from keras.layers import Conv1D, MaxPooling1D
from keras.layers.core import Dense, Dropout, Flatten
from keras.optimizers import Adam
from keras.layers.recurrent import LSTM

In [47]:
data = pd.read_csv('raw.csv', sep=',',engine='python')
df = data

In [48]:
data_new = df.dropna(subset=['ASH','INSUL','TAAV','RH','dayav_ta','dayav_rh','MET'])
data_new.to_csv('raw_new.csv')
# get experimental data
lili_new = pd.read_csv('lili_feature.csv', sep=',',engine='python')
lili = lili_new[['clo','temperature','humidity','sensation']]
lili_y = lili['sensation']
lili_x = lili[['clo','temperature','humidity']]
# get features and thermal sensation
y = data_new['ASH']
#x = data_new[['INSUL','TAAV','RH','dayav_ta','dayav_rh','MET']]
x = data_new[['INSUL','TAAV','RH']]
y = np.round(y)
from sklearn import preprocessing
lab_enc = preprocessing.LabelEncoder()
encoded = lab_enc.fit_transform(y)
lili_encoded = lab_enc.fit_transform(lili_y)
y_train = np_utils.to_categorical(encoded,7)
xx = x.values.reshape(x.shape[0],x.shape[1],1)
lili_xx = lili_x.values.reshape(lili_x.shape[0],lili_x.shape[1],1)

In [49]:
data_new = df.dropna(subset=['ASH','INSUL','TAAV','RH','dayav_ta','dayav_rh','MET'])
data_new.to_csv('raw_new.csv')
# get features and thermal sensation
y = data_new['ASH']
x = data_new[['INSUL','TAAV','RH','dayav_ta','dayav_rh','MET']]
y = np.round(y)
from sklearn import preprocessing
lab_enc = preprocessing.LabelEncoder()
encoded = lab_enc.fit_transform(y)
y_train = np_utils.to_categorical(encoded,7)
xx = x.values.reshape(x.shape[0],x.shape[1],1)

# LSTM

In [50]:
from keras.utils.vis_utils import plot_model
nb_classes=7
#input_shape=x.iloc[0].shape
input_shape=xx[0].shape
model = Sequential()
model.add(LSTM(2048,return_sequences=True,input_shape=input_shape,dropout=0.5))
model.add(Dense(512,activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(512,activation='relu'))
model.add(Dropout(0.5))
model.add(Flatten())
model.add(Dense(nb_classes, activation='sigmoid'))
#model.add(Dense(512,activation='relu',input_shape=input_shape))

In [22]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 6, 2048)           16793600  
_________________________________________________________________
dense_5 (Dense)              (None, 6, 512)            1049088   
_________________________________________________________________
dropout_5 (Dropout)          (None, 6, 512)            0         
_________________________________________________________________
dense_6 (Dense)              (None, 6, 512)            262656    
_________________________________________________________________
dropout_6 (Dropout)          (None, 6, 512)            0         
_________________________________________________________________
flatten_3 (Flatten)          (None, 3072)              0         
_________________________________________________________________
dense_7 (Dense)              (None, 7)                 21511     
Total para

In [51]:
optimizer = Adam(lr=1e-5,decay=1e-6)
model.compile(loss='categorical_crossentropy',optimizer=optimizer,metrics=['accuracy'])

In [52]:
model.fit(xx,y_train,validation_split=0.1,epochs=1)

Train on 3382 samples, validate on 376 samples
Epoch 1/1
3382/3382 [==============================] - 132s 39ms/step - loss: 1.8353 - acc: 0.4367 - val_loss: 1.4811 - val_acc: 0.5691


# CNN

In [43]:
nb_classes=7
input_shape=xx[0].shape
model = Sequential()
model.add(Conv1D(32, kernel_size=3,
                 activation='relu',
                 input_shape=input_shape))
model.add(MaxPooling1D(pool_size=2))
model.add(Dropout(0.5))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(nb_classes, activation='sigmoid'))

In [44]:
optimizer = Adam(lr=1e-5,decay=1e-6)
model.compile(loss='categorical_crossentropy',optimizer=optimizer,metrics=['accuracy'])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_5 (Conv1D)            (None, 4, 32)             128       
_________________________________________________________________
max_pooling1d_5 (MaxPooling1 (None, 2, 32)             0         
_________________________________________________________________
dropout_11 (Dropout)         (None, 2, 32)             0         
_________________________________________________________________
flatten_6 (Flatten)          (None, 64)                0         
_________________________________________________________________
dense_12 (Dense)             (None, 128)               8320      
_________________________________________________________________
dropout_12 (Dropout)         (None, 128)               0         
_________________________________________________________________
dense_13 (Dense)             (None, 7)                 903       
Total para

In [45]:
model.fit(xx,y_train,validation_split=0.1,epochs=1)

Train on 3382 samples, validate on 376 samples
Epoch 1/1
3382/3382 [==============================] - 1s 386us/step - loss: 6.0771 - acc: 0.1496 - val_loss: 4.3018 - val_acc: 0.0186


In [53]:
y_test = model.predict(xx)
pred = []
for i in range(len(y)):
    pred.append(np.argmax(y_test[i]))

In [54]:
acc = 0
for i in range(len(y)):
    if pred[i] == encoded[i]:
        acc += 1

In [55]:
accuracy = acc/len(encoded)
accuracy

0.48669505055880785

In [31]:
neutral_len_acc = len(encoded[np.where(encoded==3)])/len(encoded)
neutral_len_acc

0.48669505055880785

In [32]:
np.unique(pred)

array([6])

In [56]:
from sklearn.metrics import f1_score
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score

In [58]:
f1=f1_score(encoded, pred, average='weighted') 
r1=recall_score(encoded, pred, average='weighted') 
p1=precision_score(encoded,pred,average='weighted')
print(f1)
print(r1)
print(p1)

0.31865589671453715
0.48669505055880785
0.23687207223844056


/Users/lusiliang93/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/Users/lusiliang93/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
